In [1]:
# #!/usr/bin/python3

%matplotlib inline
%config InlineBackend.figure_formats = ["retina"]
# %load_ext autotime

#The features are borrowed from Eike Dehling's kernel at  https://www.kaggle.com/eikedehling/feature-engineering

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
from tqdm import tqdm
import gc
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
import multiprocessing

# Display progress logs on stdout
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')

In [6]:
dtypes = {
    'comment_text'   : np.unicode ,
    'toxic':         np.int16, 
    'severe_toxic': np.int16,
    'obscene': np.int16,
    'threat': np.int16,
    'insult': np.int16,
    'identity_hate': np.int16
}

train = pd.read_csv('data/train.csv', dtype=dtypes, encoding='utf-8')
test = pd.read_csv('data/test.csv', dtype=dtypes, encoding='utf-8')

train.comment_text.fillna("unknown", inplace=True)
test.comment_text.fillna("unknown",  inplace=True)

In [3]:
train_mes, valid_mes, train_l, valid_l = train_test_split(train['comment_text'],
                                                          train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
                                                          test_size=0.2, random_state=42)

In [4]:
#Using the tokenize function from Jeremy's kernel
import re, string
re_tok = re.compile(u'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s):
    return re_tok.sub(r' \1 ', s).split()

transform_com = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(train['comment_text'])

comments_train = transform_com.transform(train_mes)
comments_valid = transform_com.transform(valid_mes)
comments_test = transform_com.transform(test['comment_text'])
gc.collect()

train_mes = pd.DataFrame(train_mes)
valid_mes = pd.DataFrame(valid_mes)
data = [train_mes, valid_mes, test]

In [9]:
# These features are borrowed from https://www.kaggle.com/eikedehling/feature-engineering
for element in data:
    element['total_length'] = element['comment_text'].apply(len)
    element['capitals'] = element['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    element['caps_vs_length'] = element.apply(lambda row: float(row['capitals'])/float(row['total_length']), axis=1)
    element['num_exclamation_marks'] = element['comment_text'].apply(lambda comment: comment.count('!'))
    element['num_question_marks'] = element['comment_text'].apply(lambda comment: comment.count('?'))
    element['num_punctuation'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
    element['num_symbols'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
    element['num_words'] = element['comment_text'].apply(lambda comment: len(comment.split()))
    element['num_unique_words'] = element['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
    element['words_vs_unique'] = element['num_unique_words'] / element['num_words']
    element['num_smilies'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
    
col = ['total_length', 'capitals', 'caps_vs_length',
       'num_exclamation_marks', 'num_question_marks', 'num_punctuation',
       'num_symbols', 'num_words', 'num_unique_words', 'words_vs_unique',
       'num_smilies']
columns = ('toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate')

# Now, let's Combine both the Feature Vectors and the Engineered Features

In [14]:
train_mes = scipy.sparse.csr_matrix(train_mes[col].values)
valid_mes = scipy.sparse.csr_matrix(valid_mes[col].values)
test = scipy.sparse.csr_matrix(test[col].values)


comments_train = scipy.sparse.hstack([train_mes.tocsr(),comments_train.tocsr()])
comments_valid = scipy.sparse.hstack([valid_mes,comments_valid])
comments_test = scipy.sparse.hstack([test,comments_test])

In [15]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2017, num_rounds=500):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'auc'
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'valid') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    return model 

In [16]:
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((test.shape[0], len(col)))

for i, j in enumerate(col):
    print('fit '+j)
    model = runXGB(comments_train, train_l[j], comments_valid, valid_l[j])
    preds[:,i] = model.predict(xgb.DMatrix(comments_test), ntree_limit = model.best_ntree_limit)
    gc.collect()

fit toxic
[0]	train-auc:0.77242	test-auc:0.765702
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 20 rounds.
[1]	train-auc:0.815782	test-auc:0.809515
[2]	train-auc:0.830233	test-auc:0.824112
[3]	train-auc:0.842537	test-auc:0.833436
[4]	train-auc:0.851532	test-auc:0.842671
[5]	train-auc:0.853684	test-auc:0.845302
[6]	train-auc:0.857449	test-auc:0.847592
[7]	train-auc:0.862579	test-auc:0.849572
[8]	train-auc:0.866918	test-auc:0.852981
[9]	train-auc:0.869632	test-auc:0.855417
[10]	train-auc:0.871686	test-auc:0.857329
[11]	train-auc:0.874539	test-auc:0.860705
[12]	train-auc:0.877057	test-auc:0.861877
[13]	train-auc:0.878491	test-auc:0.861911
[14]	train-auc:0.880472	test-auc:0.863216
[15]	train-auc:0.882349	test-auc:0.864087
[16]	train-auc:0.88567	test-auc:0.867465
[17]	train-auc:0.889556	test-auc:0.869415
[18]	train-auc:0.893246	test-auc:0.871862
[19]	train-auc:0.894864	test-auc:0.872676
[20]	train-auc:0.8969

[191]	train-auc:0.987448	test-auc:0.934408
[192]	train-auc:0.987509	test-auc:0.934325
[193]	train-auc:0.987708	test-auc:0.934493
[194]	train-auc:0.987807	test-auc:0.934405
[195]	train-auc:0.987865	test-auc:0.934369
[196]	train-auc:0.987988	test-auc:0.934483
[197]	train-auc:0.98814	test-auc:0.934603
[198]	train-auc:0.988197	test-auc:0.934683
[199]	train-auc:0.988302	test-auc:0.934815
[200]	train-auc:0.98837	test-auc:0.934845
[201]	train-auc:0.988402	test-auc:0.934776
[202]	train-auc:0.988572	test-auc:0.934836
[203]	train-auc:0.988609	test-auc:0.934972
[204]	train-auc:0.988696	test-auc:0.935083
[205]	train-auc:0.988818	test-auc:0.935163
[206]	train-auc:0.988875	test-auc:0.935183
[207]	train-auc:0.98895	test-auc:0.935126
[208]	train-auc:0.989019	test-auc:0.935157
[209]	train-auc:0.989059	test-auc:0.935247
[210]	train-auc:0.989135	test-auc:0.935222
[211]	train-auc:0.989273	test-auc:0.935234
[212]	train-auc:0.989439	test-auc:0.935295
[213]	train-auc:0.989553	test-auc:0.935275
[214]	train-au

[13]	train-auc:0.943353	test-auc:0.91348
[14]	train-auc:0.944051	test-auc:0.913089
[15]	train-auc:0.944211	test-auc:0.912897
[16]	train-auc:0.955572	test-auc:0.940885
[17]	train-auc:0.955691	test-auc:0.941323
[18]	train-auc:0.962142	test-auc:0.945871
[19]	train-auc:0.967778	test-auc:0.953219
[20]	train-auc:0.967777	test-auc:0.953625
[21]	train-auc:0.970079	test-auc:0.954729
[22]	train-auc:0.970785	test-auc:0.955409
[23]	train-auc:0.970955	test-auc:0.955318
[24]	train-auc:0.971681	test-auc:0.954873
[25]	train-auc:0.975182	test-auc:0.955163
[26]	train-auc:0.976377	test-auc:0.954916
[27]	train-auc:0.976363	test-auc:0.954942
[28]	train-auc:0.976381	test-auc:0.954955
[29]	train-auc:0.976696	test-auc:0.955593
[30]	train-auc:0.977971	test-auc:0.955898
[31]	train-auc:0.983087	test-auc:0.955338
[32]	train-auc:0.983905	test-auc:0.953854
[33]	train-auc:0.986247	test-auc:0.95315
[34]	train-auc:0.987517	test-auc:0.95668
[35]	train-auc:0.988089	test-auc:0.957027
[36]	train-auc:0.988446	test-auc:0.95

[99]	train-auc:0.993243	test-auc:0.95687
[100]	train-auc:0.993333	test-auc:0.956776
[101]	train-auc:0.993556	test-auc:0.957113
[102]	train-auc:0.993649	test-auc:0.957214
[103]	train-auc:0.993679	test-auc:0.957329
[104]	train-auc:0.993806	test-auc:0.957315
[105]	train-auc:0.993892	test-auc:0.95733
[106]	train-auc:0.993926	test-auc:0.957427
[107]	train-auc:0.994068	test-auc:0.957602
[108]	train-auc:0.994118	test-auc:0.957737
[109]	train-auc:0.994241	test-auc:0.957855
[110]	train-auc:0.994303	test-auc:0.958048
[111]	train-auc:0.994353	test-auc:0.957979
[112]	train-auc:0.994456	test-auc:0.957861
[113]	train-auc:0.994518	test-auc:0.957801
[114]	train-auc:0.994586	test-auc:0.957963
[115]	train-auc:0.99464	test-auc:0.958274
[116]	train-auc:0.99472	test-auc:0.958432
[117]	train-auc:0.994784	test-auc:0.958475
[118]	train-auc:0.994831	test-auc:0.958384
[119]	train-auc:0.994966	test-auc:0.958639
[120]	train-auc:0.995007	test-auc:0.958664
[121]	train-auc:0.995057	test-auc:0.958868
[122]	train-auc:

[39]	train-auc:0.980066	test-auc:0.913364
[40]	train-auc:0.980064	test-auc:0.910812
[41]	train-auc:0.980647	test-auc:0.919733
[42]	train-auc:0.980309	test-auc:0.917757
[43]	train-auc:0.981158	test-auc:0.922466
[44]	train-auc:0.983381	test-auc:0.921396
[45]	train-auc:0.984731	test-auc:0.919606
[46]	train-auc:0.984836	test-auc:0.918707
[47]	train-auc:0.985199	test-auc:0.916094
[48]	train-auc:0.988201	test-auc:0.919753
[49]	train-auc:0.991159	test-auc:0.920017
[50]	train-auc:0.991522	test-auc:0.918763
[51]	train-auc:0.993616	test-auc:0.917049
[52]	train-auc:0.993934	test-auc:0.919724
[53]	train-auc:0.995926	test-auc:0.928921
[54]	train-auc:0.996245	test-auc:0.932596
[55]	train-auc:0.996683	test-auc:0.937738
[56]	train-auc:0.996972	test-auc:0.940543
[57]	train-auc:0.997303	test-auc:0.941461
[58]	train-auc:0.997504	test-auc:0.943013
[59]	train-auc:0.997852	test-auc:0.941857
[60]	train-auc:0.998071	test-auc:0.941992
[61]	train-auc:0.998237	test-auc:0.944663
[62]	train-auc:0.998387	test-auc:0

[99]	train-auc:0.988678	test-auc:0.944324
[100]	train-auc:0.988958	test-auc:0.944406
[101]	train-auc:0.989146	test-auc:0.94471
[102]	train-auc:0.989297	test-auc:0.944776
[103]	train-auc:0.989369	test-auc:0.944956
[104]	train-auc:0.989606	test-auc:0.945048
[105]	train-auc:0.989788	test-auc:0.94487
[106]	train-auc:0.989993	test-auc:0.945303
[107]	train-auc:0.990123	test-auc:0.945383
[108]	train-auc:0.990203	test-auc:0.945477
[109]	train-auc:0.990418	test-auc:0.94583
[110]	train-auc:0.990563	test-auc:0.945996
[111]	train-auc:0.990749	test-auc:0.945914
[112]	train-auc:0.990812	test-auc:0.946006
[113]	train-auc:0.99101	test-auc:0.945958
[114]	train-auc:0.991213	test-auc:0.946349
[115]	train-auc:0.991347	test-auc:0.946729
[116]	train-auc:0.991434	test-auc:0.94683
[117]	train-auc:0.991535	test-auc:0.946927
[118]	train-auc:0.991697	test-auc:0.947019
[119]	train-auc:0.991822	test-auc:0.947268
[120]	train-auc:0.991895	test-auc:0.947253
[121]	train-auc:0.991957	test-auc:0.947365
[122]	train-auc:0

[82]	train-auc:0.997723	test-auc:0.933482
[83]	train-auc:0.997914	test-auc:0.934327
[84]	train-auc:0.998101	test-auc:0.933809
[85]	train-auc:0.998151	test-auc:0.93438
[86]	train-auc:0.998202	test-auc:0.934004
[87]	train-auc:0.998284	test-auc:0.933203
[88]	train-auc:0.998333	test-auc:0.933148
[89]	train-auc:0.998551	test-auc:0.934569
[90]	train-auc:0.998641	test-auc:0.933932
[91]	train-auc:0.998687	test-auc:0.934429
[92]	train-auc:0.99879	test-auc:0.934636
[93]	train-auc:0.998886	test-auc:0.93564
[94]	train-auc:0.998937	test-auc:0.935705
[95]	train-auc:0.998961	test-auc:0.936943
[96]	train-auc:0.998993	test-auc:0.936804
[97]	train-auc:0.999022	test-auc:0.937946
[98]	train-auc:0.999087	test-auc:0.937456
[99]	train-auc:0.999144	test-auc:0.937387
[100]	train-auc:0.999206	test-auc:0.937433
[101]	train-auc:0.999232	test-auc:0.937461
[102]	train-auc:0.999271	test-auc:0.937041
[103]	train-auc:0.999324	test-auc:0.935862
[104]	train-auc:0.999332	test-auc:0.935584
[105]	train-auc:0.999368	test-au

In [33]:
subm = pd.read_csv('data/sample_submission.csv')    
submid = pd.DataFrame({'id': subm["id"].values.astype(str)}, dtype=np.str)
submission = pd.concat([submid, pd.DataFrame(preds, columns = col)], axis=1)
submission.to_csv('submissions/sample_submission_xgb.csv', index=False)

     ### got AUC of train-auc:0.999022	valid-auc:0.937946


# Try GloVe Word Vectors

In [7]:
train_mes, valid_mes, train_l, valid_l = train_test_split(train['comment_text'],
                                                          train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
                                                          test_size=0.2, random_state=42)
train_mes = pd.DataFrame(train_mes)
valid_mes = pd.DataFrame(valid_mes)

In [23]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('../Sentiment_Model_Template/glove.840B.300d.txt', encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except:
        continue
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [04:09, 8808.55it/s] 

Found 2195884 word vectors.


In [9]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s, embeddings_index):
    words = str(s).lower()
    words = tokenize(words)
#     words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [11]:
gc.collect() # To free up some Memory

0

In [52]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x, embeddings_index) for x in tqdm(train_mes['comment_text'])]
xvalid_glove = [sent2vec(x, embeddings_index) for x in tqdm(valid_mes['comment_text'])]

xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)




  0%|          | 0/76680 [00:00<?, ?it/s]


  0%|          | 136/76680 [00:00<00:58, 1310.42it/s]


  1%|          | 392/76680 [00:00<00:39, 1921.80it/s]


  1%|          | 684/76680 [00:00<00:33, 2247.58it/s]


  1%|          | 953/76680 [00:00<00:32, 2349.12it/s]


  2%|▏         | 1202/76680 [00:00<00:31, 2376.52it/s]


  2%|▏         | 1451/76680 [00:00<00:31, 2393.55it/s]


  2%|▏         | 1715/76680 [00:00<00:30, 2428.17it/s]


  3%|▎         | 1967/76680 [00:00<00:30, 2435.60it/s]


  3%|▎         | 2233/76680 [00:00<00:30, 2459.30it/s]


  3%|▎         | 2520/76680 [00:01<00:29, 2498.95it/s]


  4%|▎         | 2810/76680 [00:01<00:29, 2534.71it/s]


  4%|▍         | 3078/76680 [00:01<00:29, 2531.88it/s]


  4%|▍         | 3368/76680 [00:01<00:28, 2557.63it/s]


  5%|▍         | 3638/76680 [00:01<00:28, 2556.87it/s]


  5%|▌         | 3918/76680 [00:01<00:28, 2572.26it/s]


  6%|▌         | 4226/76680 [00:01<00:27, 2600.90it/s]


  6%|▌         | 4513/76680 [00:01<00:27, 261

 92%|█████████▏| 70205/76680 [00:25<00:02, 2736.03it/s]


 92%|█████████▏| 70492/76680 [00:25<00:02, 2736.47it/s]


 92%|█████████▏| 70838/76680 [00:25<00:02, 2739.29it/s]


 93%|█████████▎| 71145/76680 [00:25<00:02, 2740.45it/s]


 93%|█████████▎| 71445/76680 [00:26<00:01, 2740.70it/s]


 94%|█████████▎| 71740/76680 [00:26<00:01, 2740.15it/s]


 94%|█████████▍| 72025/76680 [00:26<00:01, 2739.95it/s]


 94%|█████████▍| 72309/76680 [00:26<00:01, 2740.29it/s]


 95%|█████████▍| 72600/76680 [00:26<00:01, 2740.84it/s]


 95%|█████████▌| 72883/76680 [00:26<00:01, 2740.51it/s]


 95%|█████████▌| 73203/76680 [00:26<00:01, 2742.19it/s]


 96%|█████████▌| 73494/76680 [00:26<00:01, 2742.10it/s]


 96%|█████████▋| 73816/76680 [00:26<00:01, 2743.79it/s]


 97%|█████████▋| 74112/76680 [00:27<00:00, 2744.30it/s]


 97%|█████████▋| 74405/76680 [00:27<00:00, 2744.42it/s]


 97%|█████████▋| 74729/76680 [00:27<00:00, 2746.21it/s]


 98%|█████████▊| 75028/76680 [00:27<00:00, 2746.40it/s]


 98%|█████████

In [53]:
xtrain_glove.shape

(76680, 300)

In [54]:
xvalid_glove.shape

(19171, 300)

In [55]:
scores = []
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((valid_mes.shape[0], len(col)))

for i, class_name in enumerate(col):
    print('fit '+ class_name)
#     classifier = LogisticRegression(C=0.1, solver='sag')
    classifier = SGDClassifier(loss='log', max_iter=1000, epsilon=0.001, n_jobs=-1)
#     classifier = SVC(C=1.0, probability=True)

    cv_score = np.mean(cross_val_score(classifier, xtrain_glove, train_l[class_name], cv=5, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(xtrain_glove, train_l[class_name])
    preds[:,i] = classifier.predict_proba(xvalid_glove) 
    
print('Total CV score is {}'.format(np.mean(scores)))


fit toxic
CV score for class toxic is 0.7815914773233411
fit severe_toxic
CV score for class severe_toxic is 0.9074135380000475
fit obscene
CV score for class obscene is 0.8259538580137147
fit threat
CV score for class threat is 0.8624474536468689
fit insult
CV score for class insult is 0.8230832481127066
fit identity_hate
CV score for class identity_hate is 0.8288065779076506
Total CV score is 0.8382160255007216


In [36]:
# Generate Word vectors of test data
xtest_glove = [sent2vec(x, embeddings_index) for x in tqdm(test['comment_text'])]
xtest_glove = np.array(xtest_glove)

In [28]:
testpreds_glove = np.zeros((test.shape[0], len(col)))
for i, class_name in enumerate(col):
    print('fit '+ class_name)
    testpreds_google[:,i] = classifier.predict_proba(xtest_glove)[:,1] 

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [58]:
submission = pd.concat([submid, pd.DataFrame(testpreds_glove, columns = col)], axis=1)
submission.to_csv('submissions/sample_submission_glove_sgd.csv', index=False)

    ### got total AUC of 0.838 on validation set

# Try Google Word Vectors

In [3]:
gc.collect()

0

In [4]:
# load the GloVe vectors in a dictionary:
embeddings_index_google = {}
f = open('../Sentiment_Model_Template/GoogleNews-vectors-negative300.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float32')
    except:
        continue
    embeddings_index_google[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_google))

3000001it [03:00, 16636.95it/s]

Found 3000000 word vectors.


In [19]:
gc.collect()

0

In [14]:
# create sentence vectors using the above function for training and validation set
xtrain_google = [sent2vec(x, embeddings_index_google) for x in tqdm(train_mes['comment_text'])]
xvalid_google = [sent2vec(x, embeddings_index_google) for x in tqdm(valid_mes['comment_text'])]

xtrain_google = np.array(xtrain_google)
xvalid_google = np.array(xvalid_google)



  0%|          | 0/76680 [00:00<?, ?it/s]

  0%|          | 279/76680 [00:00<00:27, 2761.13it/s]

  1%|          | 580/76680 [00:00<00:26, 2876.78it/s]

  1%|          | 829/76680 [00:00<00:27, 2743.46it/s]

  1%|▏         | 1080/76680 [00:00<00:28, 2684.11it/s]

  2%|▏         | 1325/76680 [00:00<00:28, 2634.96it/s]

  2%|▏         | 1574/76680 [00:00<00:29, 2580.84it/s]

  2%|▏         | 1836/76680 [00:00<00:28, 2587.94it/s]

  3%|▎         | 2068/76680 [00:00<00:29, 2537.41it/s]

  3%|▎         | 2338/76680 [00:00<00:29, 2552.90it/s]

  3%|▎         | 2604/76680 [00:01<00:28, 2563.17it/s]

  4%|▎         | 2873/76680 [00:01<00:28, 2572.88it/s]

  4%|▍         | 3174/76680 [00:01<00:28, 2607.04it/s]

  5%|▍         | 3484/76680 [00:01<00:27, 2644.25it/s]

  5%|▍         | 3764/76680 [00:01<00:27, 2640.50it/s]

  5%|▌         | 4066/76680 [00:01<00:27, 2664.40it/s]

  6%|▌         | 4360/76680 [00:01<00:26, 2680.94it/s]

  6%|▌         | 4656/76680 [00:01<00:26, 2696.92it/s]

  6%|▋

 49%|████▉     | 37940/76680 [00:13<00:13, 2798.10it/s]

 50%|████▉     | 38243/76680 [00:13<00:13, 2795.70it/s]

 50%|█████     | 38546/76680 [00:13<00:13, 2797.38it/s]

 51%|█████     | 38914/76680 [00:13<00:13, 2803.74it/s]

 51%|█████     | 39242/76680 [00:13<00:13, 2807.08it/s]

 52%|█████▏    | 39561/76680 [00:14<00:13, 2806.78it/s]

 52%|█████▏    | 39868/76680 [00:14<00:13, 2805.95it/s]

 52%|█████▏    | 40187/76680 [00:14<00:12, 2808.63it/s]

 53%|█████▎    | 40513/76680 [00:14<00:12, 2811.74it/s]

 53%|█████▎    | 40823/76680 [00:14<00:12, 2811.86it/s]

 54%|█████▎    | 41152/76680 [00:14<00:12, 2815.09it/s]

 54%|█████▍    | 41462/76680 [00:14<00:12, 2816.08it/s]

 54%|█████▍    | 41768/76680 [00:14<00:12, 2816.85it/s]

 55%|█████▍    | 42070/76680 [00:14<00:12, 2817.11it/s]

 55%|█████▌    | 42386/76680 [00:15<00:12, 2819.35it/s]

 56%|█████▌    | 42710/76680 [00:15<00:12, 2821.90it/s]

 56%|█████▌    | 43018/76680 [00:15<00:11, 2817.03it/s]

 56%|█████▋    | 43302/76680 [0

  5%|▍         | 941/19171 [00:00<00:05, 3124.13it/s]

  6%|▋         | 1228/19171 [00:00<00:05, 3048.71it/s]

  8%|▊         | 1533/19171 [00:00<00:05, 3047.62it/s]

 10%|▉         | 1877/19171 [00:00<00:05, 3109.58it/s]

 11%|█▏        | 2186/19171 [00:00<00:05, 3104.82it/s]

 13%|█▎        | 2516/19171 [00:00<00:05, 3123.18it/s]

 15%|█▍        | 2822/19171 [00:00<00:05, 3114.43it/s]

 16%|█▋        | 3150/19171 [00:01<00:05, 3130.73it/s]

 18%|█▊        | 3459/19171 [00:01<00:05, 3097.15it/s]

 20%|█▉        | 3760/19171 [00:01<00:04, 3089.86it/s]

 21%|██        | 4061/19171 [00:01<00:04, 3053.93it/s]

 23%|██▎       | 4352/19171 [00:01<00:04, 3033.56it/s]

 24%|██▍       | 4639/19171 [00:01<00:04, 3018.60it/s]

 26%|██▌       | 4931/19171 [00:01<00:04, 3012.40it/s]

 27%|██▋       | 5242/19171 [00:01<00:04, 3016.64it/s]

 29%|██▉       | 5536/19171 [00:01<00:04, 2963.10it/s]

 30%|███       | 5838/19171 [00:01<00:04, 2965.99it/s]

 32%|███▏      | 6124/19171 [00:02<00:04, 2960.69

In [15]:
xtrain_google.shape

(76680, 300)

In [16]:
xvalid_google.shape

(19171, 300)

In [22]:
scores = []
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds_google = np.zeros((valid_mes.shape[0], len(col)))

for i, class_name in enumerate(col):
    print('fit '+ class_name)
    classifier = SGDClassifier(alpha= 1e-05, max_iter= 10000, loss='log', penalty= 'l2', n_jobs=-1, random_state=42)

    cv_score = np.mean(cross_val_score(classifier, xtrain_google, train_l[class_name], cv=5, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(xtrain_google, train_l[class_name])
    preds_google[:,i] = classifier.predict_proba(xvalid_google)[:,1] 
    
print('Total CV score is {}'.format(np.mean(scores)))


fit toxic
CV score for class toxic is 0.8010249179465745
fit severe_toxic
CV score for class severe_toxic is 0.9250436307925302
fit obscene
CV score for class obscene is 0.8458443652900014
fit threat
CV score for class threat is 0.8794754857125302
fit insult
CV score for class insult is 0.8432770129647181
fit identity_hate
CV score for class identity_hate is 0.8365719828043383
Total CV score is 0.8552062325851154


In [25]:
gc.collect()

0

In [26]:
# Generate Word vectors of test data
xtest_google = [sent2vec(x, embeddings_index_google) for x in tqdm(test['comment_text'])]
xtest_google = np.array(xtest_google)



  0%|          | 0/226998 [00:00<?, ?it/s]

  0%|          | 1/226998 [00:00<26:15:24,  2.40it/s]

  0%|          | 2/226998 [00:01<32:35:42,  1.93it/s]

  0%|          | 3/226998 [00:01<30:20:40,  2.08it/s]

  0%|          | 6/226998 [00:02<23:46:27,  2.65it/s]

  0%|          | 7/226998 [00:02<22:14:41,  2.83it/s]

  0%|          | 10/226998 [00:02<16:42:23,  3.77it/s]

  0%|          | 11/226998 [00:02<15:48:00,  3.99it/s]

  0%|          | 12/226998 [00:02<15:07:11,  4.17it/s]

  0%|          | 15/226998 [00:03<12:44:37,  4.95it/s]

  0%|          | 17/226998 [00:03<12:35:15,  5.01it/s]

  0%|          | 19/226998 [00:03<11:38:20,  5.42it/s]

  0%|          | 21/226998 [00:03<11:14:50,  5.61it/s]

  0%|          | 23/226998 [00:03<10:35:51,  5.95it/s]

  0%|          | 25/226998 [00:03<10:00:59,  6.29it/s]

  0%|          | 27/226998 [00:04<9:43:35,  6.48it/s] 

  0%|          | 32/226998 [00:04<8:28:00,  7.45it/s]

  0%|          | 35/226998 [00:04<8:02:21,  7.84it/s]

  0%|    

  0%|          | 816/226998 [00:22<1:41:40, 37.07it/s]

  0%|          | 825/226998 [00:22<1:41:02, 37.31it/s]

  0%|          | 834/226998 [00:22<1:40:34, 37.48it/s]

  0%|          | 852/226998 [00:22<1:39:07, 38.03it/s]

  0%|          | 861/226998 [00:22<1:38:39, 38.20it/s]

  0%|          | 872/226998 [00:22<1:37:53, 38.50it/s]

  0%|          | 881/226998 [00:22<1:37:44, 38.55it/s]

  0%|          | 889/226998 [00:22<1:37:16, 38.74it/s]

  0%|          | 898/226998 [00:23<1:36:50, 38.91it/s]

  0%|          | 910/226998 [00:23<1:35:55, 39.28it/s]

  0%|          | 920/226998 [00:23<1:35:24, 39.49it/s]

  0%|          | 932/226998 [00:23<1:34:34, 39.84it/s]

  0%|          | 942/226998 [00:23<1:34:20, 39.93it/s]

  0%|          | 951/226998 [00:23<1:34:08, 40.02it/s]

  0%|          | 961/226998 [00:23<1:33:34, 40.26it/s]

  0%|          | 976/226998 [00:23<1:32:31, 40.71it/s]

  0%|          | 990/226998 [00:24<1:31:38, 41.10it/s]

  0%|          | 1001/226998 [00:24<1:31:17, 41.

  1%|          | 2075/226998 [00:39<1:12:13, 51.90it/s]

  1%|          | 2086/226998 [00:40<1:12:02, 52.03it/s]

  1%|          | 2094/226998 [00:40<1:11:58, 52.08it/s]

  1%|          | 2111/226998 [00:40<1:11:38, 52.32it/s]

  1%|          | 2120/226998 [00:40<1:11:33, 52.38it/s]

  1%|          | 2134/226998 [00:40<1:11:16, 52.59it/s]

  1%|          | 2144/226998 [00:40<1:11:09, 52.66it/s]

  1%|          | 2157/226998 [00:40<1:10:55, 52.84it/s]

  1%|          | 2168/226998 [00:40<1:10:43, 52.98it/s]

  1%|          | 2179/226998 [00:41<1:10:35, 53.08it/s]

  1%|          | 2190/226998 [00:41<1:10:24, 53.21it/s]

  1%|          | 2201/226998 [00:41<1:10:15, 53.33it/s]

  1%|          | 2211/226998 [00:41<1:10:08, 53.42it/s]

  1%|          | 2225/226998 [00:41<1:09:52, 53.62it/s]

  1%|          | 2236/226998 [00:41<1:09:44, 53.71it/s]

  1%|          | 2248/226998 [00:41<1:09:33, 53.86it/s]

  1%|          | 2262/226998 [00:41<1:09:16, 54.06it/s]

  1%|          | 2274/226998 [0

  2%|▏         | 3639/226998 [00:57<58:54, 63.20it/s]

  2%|▏         | 3652/226998 [00:57<58:47, 63.32it/s]

  2%|▏         | 3662/226998 [00:57<58:48, 63.30it/s]

  2%|▏         | 3672/226998 [00:57<58:45, 63.35it/s]

  2%|▏         | 3681/226998 [00:58<58:42, 63.39it/s]

  2%|▏         | 3690/226998 [00:58<58:40, 63.43it/s]

  2%|▏         | 3699/226998 [00:58<58:41, 63.41it/s]

  2%|▏         | 3707/226998 [00:58<58:41, 63.41it/s]

  2%|▏         | 3715/226998 [00:58<58:39, 63.44it/s]

  2%|▏         | 3728/226998 [00:58<58:32, 63.56it/s]

  2%|▏         | 3737/226998 [00:58<58:32, 63.57it/s]

  2%|▏         | 3750/226998 [00:58<58:26, 63.67it/s]

  2%|▏         | 3760/226998 [00:59<58:25, 63.69it/s]

  2%|▏         | 3770/226998 [00:59<58:21, 63.74it/s]

  2%|▏         | 3779/226998 [00:59<58:20, 63.76it/s]

  2%|▏         | 3788/226998 [00:59<58:19, 63.78it/s]

  2%|▏         | 3799/226998 [00:59<58:14, 63.87it/s]

  2%|▏         | 3808/226998 [00:59<58:12, 63.91it/s]

  2%|▏    

  2%|▏         | 4943/226998 [01:15<56:13, 65.82it/s]

  2%|▏         | 4957/226998 [01:15<56:10, 65.89it/s]

  2%|▏         | 4973/226998 [01:15<56:06, 65.95it/s]

  2%|▏         | 4986/226998 [01:15<56:03, 66.01it/s]

  2%|▏         | 4998/226998 [01:15<55:59, 66.08it/s]

  2%|▏         | 5011/226998 [01:15<55:54, 66.17it/s]

  2%|▏         | 5025/226998 [01:15<55:50, 66.25it/s]

  2%|▏         | 5039/226998 [01:15<55:45, 66.34it/s]

  2%|▏         | 5052/226998 [01:16<55:42, 66.39it/s]

  2%|▏         | 5066/226998 [01:16<55:38, 66.48it/s]

  2%|▏         | 5079/226998 [01:16<55:34, 66.56it/s]

  2%|▏         | 5091/226998 [01:16<55:34, 66.55it/s]

  2%|▏         | 5103/226998 [01:16<55:30, 66.62it/s]

  2%|▏         | 5119/226998 [01:16<55:24, 66.73it/s]

  2%|▏         | 5131/226998 [01:16<55:23, 66.75it/s]

  2%|▏         | 5174/226998 [01:16<55:00, 67.21it/s]

  2%|▏         | 5214/226998 [01:17<54:38, 67.64it/s]

  2%|▏         | 5241/226998 [01:17<54:25, 67.91it/s]

  2%|▏    

...

 84%|████████▍ | 191295/226998 [09:54<01:50, 322.01it/s]

 84%|████████▍ | 191371/226998 [09:54<01:50, 322.07it/s]

 84%|████████▍ | 191442/226998 [09:54<01:50, 322.12it/s]

 84%|████████▍ | 191523/226998 [09:54<01:50, 322.20it/s]

 84%|████████▍ | 191594/226998 [09:54<01:49, 322.26it/s]

 84%|████████▍ | 191664/226998 [09:54<01:49, 322.32it/s]

 84%|████████▍ | 191733/226998 [09:54<01:49, 322.35it/s]

 84%|████████▍ | 191794/226998 [09:54<01:49, 322.39it/s]

 85%|████████▍ | 191893/226998 [09:55<01:48, 322.50it/s]

 85%|████████▍ | 191963/226998 [09:55<01:48, 322.56it/s]

 85%|████████▍ | 192067/226998 [09:55<01:48, 322.68it/s]

 85%|████████▍ | 192146/226998 [09:55<01:47, 322.75it/s]

 85%|████████▍ | 192223/226998 [09:55<01:47, 322.82it/s]

 85%|████████▍ | 192297/226998 [09:55<01:47, 322.87it/s]

 85%|████████▍ | 192366/226998 [09:55<01:47, 322.90it/s]

 85%|████████▍ | 192436/226998 [09:55<01:47, 322.96it/s]

 85%|████████▍ | 192500/226998 [09:55<01:46, 323.00it/s]

 85%|████████▍

 88%|████████▊ | 198814/226998 [10:09<01:26, 326.28it/s]

 88%|████████▊ | 198870/226998 [10:09<01:26, 326.29it/s]

 88%|████████▊ | 198932/226998 [10:09<01:26, 326.33it/s]

 88%|████████▊ | 198985/226998 [10:09<01:25, 326.34it/s]

 88%|████████▊ | 199034/226998 [10:09<01:25, 326.31it/s]

 88%|████████▊ | 199075/226998 [10:10<01:25, 326.31it/s]

 88%|████████▊ | 199137/226998 [10:10<01:25, 326.35it/s]

 88%|████████▊ | 199182/226998 [10:10<01:25, 326.37it/s]

 88%|████████▊ | 199236/226998 [10:10<01:25, 326.41it/s]

 88%|████████▊ | 199296/226998 [10:10<01:24, 326.45it/s]

 88%|████████▊ | 199347/226998 [10:10<01:24, 326.47it/s]

 88%|████████▊ | 199410/226998 [10:10<01:24, 326.52it/s]

 88%|████████▊ | 199471/226998 [10:10<01:24, 326.56it/s]

 88%|████████▊ | 199525/226998 [10:10<01:24, 326.58it/s]

 88%|████████▊ | 199602/226998 [10:11<01:23, 326.65it/s]

 88%|████████▊ | 199660/226998 [10:11<01:23, 326.68it/s]

 88%|████████▊ | 199716/226998 [10:11<01:23, 326.68it/s]

 88%|████████▊

 91%|█████████ | 206392/226998 [10:25<01:02, 330.10it/s]

 91%|█████████ | 206450/226998 [10:25<01:02, 330.14it/s]

 91%|█████████ | 206506/226998 [10:25<01:02, 330.16it/s]

 91%|█████████ | 206557/226998 [10:25<01:01, 330.15it/s]

 91%|█████████ | 206628/226998 [10:25<01:01, 330.21it/s]

 91%|█████████ | 206685/226998 [10:25<01:01, 330.24it/s]

 91%|█████████ | 206737/226998 [10:25<01:01, 330.26it/s]

 91%|█████████ | 206794/226998 [10:26<01:01, 330.30it/s]

 91%|█████████ | 206846/226998 [10:26<01:01, 330.33it/s]

 91%|█████████ | 206914/226998 [10:26<01:00, 330.38it/s]

 91%|█████████ | 206973/226998 [10:26<01:00, 330.42it/s]

 91%|█████████ | 207029/226998 [10:26<01:00, 330.45it/s]

 91%|█████████ | 207088/226998 [10:26<01:00, 330.49it/s]

 91%|█████████▏| 207145/226998 [10:26<01:00, 330.53it/s]

 91%|█████████▏| 207202/226998 [10:26<00:59, 330.57it/s]

 91%|█████████▏| 207281/226998 [10:26<00:59, 330.64it/s]

 91%|█████████▏| 207344/226998 [10:27<00:59, 330.65it/s]

 91%|█████████

 95%|█████████▍| 215323/226998 [10:40<00:34, 336.04it/s]

 95%|█████████▍| 215397/226998 [10:40<00:34, 336.10it/s]

 95%|█████████▍| 215492/226998 [10:40<00:34, 336.19it/s]

 95%|█████████▍| 215569/226998 [10:41<00:33, 336.24it/s]

 95%|█████████▍| 215642/226998 [10:41<00:33, 336.29it/s]

 95%|█████████▌| 215721/226998 [10:41<00:33, 336.36it/s]

 95%|█████████▌| 215810/226998 [10:41<00:33, 336.44it/s]

 95%|█████████▌| 215887/226998 [10:41<00:33, 336.50it/s]

 95%|█████████▌| 216012/226998 [10:41<00:32, 336.65it/s]

 95%|█████████▌| 216116/226998 [10:41<00:32, 336.75it/s]

 95%|█████████▌| 216209/226998 [10:41<00:32, 336.82it/s]

 95%|█████████▌| 216304/226998 [10:42<00:31, 336.91it/s]

 95%|█████████▌| 216388/226998 [10:42<00:31, 336.95it/s]

 95%|█████████▌| 216465/226998 [10:42<00:31, 337.03it/s]

 95%|█████████▌| 216539/226998 [10:42<00:31, 337.04it/s]

 95%|█████████▌| 216626/226998 [10:42<00:30, 337.12it/s]

 95%|█████████▌| 216712/226998 [10:42<00:30, 337.20it/s]

 96%|█████████

 99%|█████████▊| 223634/226998 [10:56<00:09, 340.90it/s]

 99%|█████████▊| 223713/226998 [10:56<00:09, 340.97it/s]

 99%|█████████▊| 223771/226998 [10:56<00:09, 341.00it/s]

 99%|█████████▊| 223828/226998 [10:56<00:09, 341.00it/s]

 99%|█████████▊| 223879/226998 [10:56<00:09, 341.00it/s]

 99%|█████████▊| 223924/226998 [10:56<00:09, 341.00it/s]

 99%|█████████▊| 223981/226998 [10:56<00:08, 341.04it/s]

 99%|█████████▊| 224029/226998 [10:56<00:08, 341.01it/s]

 99%|█████████▊| 224095/226998 [10:57<00:08, 341.06it/s]

 99%|█████████▊| 224148/226998 [10:57<00:08, 341.08it/s]

 99%|█████████▉| 224195/226998 [10:57<00:08, 341.07it/s]

 99%|█████████▉| 224268/226998 [10:57<00:08, 341.13it/s]

 99%|█████████▉| 224320/226998 [10:57<00:07, 341.15it/s]

 99%|█████████▉| 224373/226998 [10:57<00:07, 341.16it/s]

 99%|█████████▉| 224431/226998 [10:57<00:07, 341.21it/s]

 99%|█████████▉| 224495/226998 [10:57<00:07, 341.25it/s]

 99%|█████████▉| 224551/226998 [10:57<00:07, 341.28it/s]

 99%|█████████

In [28]:
testpreds_google = np.zeros((test.shape[0], len(col)))
for i, class_name in enumerate(col):
    print('fit '+ class_name)
    testpreds_google[:,i] = classifier.predict_proba(xtest_google)[:,1] 

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [34]:
submission = pd.concat([submid, pd.DataFrame(testpreds_google, columns = col)], axis=1)
submission.to_csv('submissions/sample_submission_google_sgd.csv', index=False)